In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path_1 = "/content/drive/My Drive/IMDDataSample1deg_1981_2021.npy"
file_path_2 = "/content/drive/My Drive/BiasDataSample1deg_1981_2021.npy"
data_1 = np.load(file_path_1)
data_2 = np.load(file_path_2)

In [4]:
data_1.shape

(2132, 36, 36)

In [5]:
data_2.shape

(2132, 36, 36)

In [6]:
data_1

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
data_2

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
observational_data = data_1[:1716, :, :]
GCM_data=data_2[:1716,:,:]

In [9]:
GCM_data = GCM_data.reshape(-1, 36, 36, 1)
observational_data = observational_data.reshape(-1, 36, 36, 1)

In [10]:
scaler = StandardScaler()
GCM_data = scaler.fit_transform(GCM_data.reshape(-1, 1)).reshape(GCM_data.shape)
observational_data = scaler.transform(observational_data.reshape(-1, 1)).reshape(observational_data.shape)

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(GCM_data, observational_data, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [29]:
from keras.layers import Add
from keras.models import Model

input_layer = keras.layers.Input(shape=(36, 36, 1))

# Feature extraction layers
x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

# Residual blocks
for _ in range(8):  # Experiment with the number of residual blocks
    residual = x
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = Add()([residual, x])  # Residual connection

# Output layer
output_layer = keras.layers.Conv2D(1, (3, 3), activation='linear', padding='same')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)


In [30]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = keras.callbacks.ModelCheckpoint('bias_correction_model.h5', save_best_only=True)

In [32]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
38/38 [==============================] - 2s 65ms/step - loss: 0.0614 - mean_absolute_error: 0.1356 - val_loss: 0.0627 - val_mean_absolute_error: 0.1318
Epoch 2/100
38/38 [==============================] - 2s 58ms/step - loss: 0.0687 - mean_absolute_error: 0.1532 - val_loss: 0.0754 - val_mean_absolute_error: 0.1706
Epoch 3/100
38/38 [==============================] - 2s 64ms/step - loss: 0.0678 - mean_absolute_error: 0.1558 - val_loss: 0.0620 - val_mean_absolute_error: 0.1350
Epoch 4/100
38/38 [==============================] - 2s 59ms/step - loss: 0.0643 - mean_absolute_error: 0.1440 - val_loss: 0.0680 - val_mean_absolute_error: 0.1533
Epoch 5/100
38/38 [==============================] - 2s 61ms/step - loss: 0.0653 - mean_absolute_error: 0.1443 - val_loss: 0.0660 - val_mean_absolute_error: 0.1465
Epoch 6/100
38/38 [==============================] - 2s 64ms/step - loss: 0.0631 - mean_absolute_error: 0.1396 - val_loss: 0.0619 - val_mean_absolute_error: 0.1327
Epoch 7/100
38/3

In [33]:
best_model = keras.models.load_model('bias_correction_model.h5')

In [34]:
test_loss, test_mae = best_model.evaluate(X_test, y_test)

9/9 [==============================] - 1s 26ms/step - loss: 0.0649 - mean_absolute_error: 0.1351


In [35]:
bias_corrected_data = best_model.predict(GCM_data)

54/54 [==============================] - 1s 18ms/step


In [36]:
bias_corrected_data = bias_corrected_data.reshape(-1, 36, 36)

In [37]:
GCM_data_real=GCM_data.reshape(-1,36,36)

In [38]:
differences=bias_corrected_data-GCM_data_real

In [39]:
mean_differences=np.mean(differences,axis=(1,2))
global_mean_difference=np.mean(differences)
print(global_mean_difference)

0.2509495
